In [1]:
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  
from matplotlib import cm
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [3]:
#Libraries
from collections import Counter
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  
from matplotlib import cm
import statsmodels.api as sm
import statsmodels.stats.weightstats as sms
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LogisticRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer,LabelEncoder 
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV 
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestRegressor, AdaBoostRegressor 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import roc_auc_score
import pandas as pd
%matplotlib inline
## change the number of columns visible in the notebook 
pd.set_option('max_columns', 99)

In [4]:
final = pd.read_csv('final_data_for_modeling.csv')

In [5]:
final.head()

,tweet,class,date_posted
0,If you are heading to and beyond going SB on ...,0,2017-12-13 23:22:50+00:00
1,"Due to bridge repairs, TxDOT has closed two al...",0,2017-12-13 01:35:30+00:00
2,Good news. All mainlanes of I-45 N Frwy NB bet...,0,2017-12-04 02:46:13+00:00
3,I-45 Northbound direct connector project conti...,0,2017-12-01 14:49:35+00:00
4,Two incident on I-10 Katy Frwy are significant...,0,2017-11-28 22:18:29+00:00


In [6]:
final.shape

(15476, 3)

In [7]:
final.drop_duplicates(keep='first', inplace=True)

In [8]:
final.shape

(14513, 3)

In [9]:
X = final['tweet'] # features
y = final['class'] # targettweets

In [10]:
import spacy

In [11]:
nlp = spacy.load('en_core_web_sm')

In [12]:
tweet1 = nlp(X.loc[0])

In [13]:
tweet1

If you are heading to  and beyond going SB on I-69/ US 59 - multiple lanes will be closed immediately for emergency road repair. Seek alt route and avoid the area. Closure may last until morning hours. HOUTraffic

In [14]:
for token in tweet1:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_, token.dep_, token.head.text)

If SCONJ mark heading
you PRON nsubj heading
are AUX aux heading
heading VERB advcl closed
to ADP prep heading
  SPACE  to
and CCONJ cc to
beyond ADP conj to
going VERB pcomp beyond
SB PROPN advmod going
on ADP prep going
I-69/ PROPN pobj on
US PROPN nmod lanes
59 NUM nummod multiple
- PUNCT punct multiple
multiple ADJ amod lanes
lanes NOUN nsubjpass closed
will VERB aux closed
be AUX auxpass closed
closed VERB ROOT closed
immediately ADV advmod closed
for ADP prep closed
emergency NOUN compound repair
road NOUN compound repair
repair NOUN pobj for
. PUNCT punct closed
Seek VERB ROOT Seek
alt NOUN compound route
route NOUN dobj Seek
and CCONJ cc Seek
avoid VERB conj Seek
the DET det area
area NOUN dobj avoid
. PUNCT punct Seek
Closure NOUN nsubj last
may VERB aux last
last VERB ROOT last
until ADP prep last
morning NOUN compound hours
hours NOUN pobj until
. PUNCT punct last
HOUTraffic PROPN ROOT HOUTraffic


In [15]:
for ent in tweet1.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

US GPE
59 CARDINAL
morning hours TIME


In [16]:
spacy.explain('GPE')

'Countries, cities, states'

In [17]:
spacy.explain('CARDINAL')

'Numerals that do not fall under another type'

In [18]:
spacy.explain('TIME')

'Times smaller than a day'

In [19]:
## import the data
final = pd.read_csv('final_data_for_modeling.csv')

In [20]:
final.head()

,tweet,class,date_posted
0,If you are heading to and beyond going SB on ...,0,2017-12-13 23:22:50+00:00
1,"Due to bridge repairs, TxDOT has closed two al...",0,2017-12-13 01:35:30+00:00
2,Good news. All mainlanes of I-45 N Frwy NB bet...,0,2017-12-04 02:46:13+00:00
3,I-45 Northbound direct connector project conti...,0,2017-12-01 14:49:35+00:00
4,Two incident on I-10 Katy Frwy are significant...,0,2017-11-28 22:18:29+00:00


In [21]:
# creating a dataframe of only closure tweets
closure_tweets = final[final['class'] == 0]

In [22]:
# inspecting the closure tweets
print(closure_tweets.shape)
closure_tweets.head()

(7738, 3)


,tweet,class,date_posted
0,If you are heading to and beyond going SB on ...,0,2017-12-13 23:22:50+00:00
1,"Due to bridge repairs, TxDOT has closed two al...",0,2017-12-13 01:35:30+00:00
2,Good news. All mainlanes of I-45 N Frwy NB bet...,0,2017-12-04 02:46:13+00:00
3,I-45 Northbound direct connector project conti...,0,2017-12-01 14:49:35+00:00
4,Two incident on I-10 Katy Frwy are significant...,0,2017-11-28 22:18:29+00:00


In [25]:
# to use spaCy
# need to make changes to the tweets
# e.g. change 'Frwy' in the fifth tweet in the dataframe to "freeway"

traffic_dict = {'US ' : 'US-', 
                'I-' : 'Interstate ',  
                'CR ' : 'Country Route ',
                'Hwy' : 'Highway ',
                'St ' : 'Street ',
                'St. ' : 'Street ',
                'Dr': 'Drive',
                'Dr.': 'Drive',
                'Rt ' : 'Route ',
                'Rd' : 'Road ',
                'Rd. ' : 'Road ',
                'Rte ' : 'Route ',
                'SB' : 'southbound', 
                'NB' : 'northbound', 
                'WB': 'westbound',
                'EB': 'eastbound',
               'Frwy': 'freeway'
               }

In [26]:
clear_tweets = []
for clean in closure_tweets['tweet']:
    for key in traffic_dict:
        cleaned = clean.replace(key, traffic_dict[key])
    clear_tweets.append(cleaned)
closure_tweets['cleaner_tweet'] = clear_tweets

In [27]:
closure_tweets

,tweet,class,date_posted,cleaner_tweet
0,If you are heading to and beyond going SB on ...,0,2017-12-13 23:22:50+00:00,If you are heading to and beyond going SB on ...
1,"Due to bridge repairs, TxDOT has closed two al...",0,2017-12-13 01:35:30+00:00,"Due to bridge repairs, TxDOT has closed two al..."
2,Good news. All mainlanes of I-45 N Frwy NB bet...,0,2017-12-04 02:46:13+00:00,Good news. All mainlanes of I-45 N freeway NB ...
3,I-45 Northbound direct connector project conti...,0,2017-12-01 14:49:35+00:00,I-45 Northbound direct connector project conti...
4,Two incident on I-10 Katy Frwy are significant...,0,2017-11-28 22:18:29+00:00,Two incident on I-10 Katy freeway are signific...
...,...,...,...,...
7733,PolarVortex: It's so cold in Wisconsin that th...,0,2019-01-30 13:51:14+00:00,PolarVortex: It's so cold in Wisconsin that th...
7734,We have another chance to see some patchy fog ...,0,2019-01-06 23:50:44+00:00,We have another chance to see some patchy fog ...
7735,Close to normal lows for this time of the year...,0,2019-12-22 11:54:01+00:00,Close to normal lows for this time of the year...
7736,2 direct connectors at I-10 and I-69 interchan...,0,2019-11-01 00:15:01+00:00,2 direct connectors at I-10 and I-69 interchan...


In [28]:
# choosing random data to use in named entity recognition training module

test_col = closure_tweets['cleaner_tweet'].sample(n = 45, replace = False)

In [29]:
import numpy as np

In [30]:
np.savetxt('spacy_annotator_test.txt', test_col, fmt='%s')

In [31]:
## label data

In [32]:
# used 40 randomly selected tweets above to train out model on name recognition.
# https://manivannanmurugavel.github.io/annotating-tool/spacy-ner-annotator/
# https://medium.com/@manivannan_data/how-to-train-ner-with-custom-training-data-using-spacy-188e0e508c6
# output is a json file

In [34]:
import json

ner_trained = input('/Houston NER.json')

with open(ner_trained) as train_data:
    train = json.load(train_data)
    
training_data_ner = []
for data in train:
    ents = [tuple(entity) for entity in data['entities']]
    training_data_ner.append((data['content'],{'entities':ents}))


with open('{}'.format(ner_trained.replace('json','txt')),'w') as write:
    write.write(str(training_data_ner))

/Houston NER.jsonHouston NER.json


In [35]:
type(training_data_ner)

list

In [36]:
ner_train

NameError: name 'ner_train' is not defined

In [37]:
# inspecting our converted data and confirming that it is now a python list
training_data_ner

[('Additional road closures in southeast Houston: 1) Airport between Telephone Road and Broadway 2) Mykawa between Dixie and Interstate Highway 610 3) Wayside between Dixie and Interstate Highway 610 4) Cullen at Almeda Genoa 5) Scott Street at Interstate Highway 610 houwx',
  {'entities': [(97, 144, 'relative location'),
    (50, 94, 'relative location'),
    (148, 196, 'relative location'),
    (226, 264, 'relative location'),
    (199, 222, 'relative location'),
    (28, 45, 'region')]}),
 ('Closed due to accident in SouthSide on Hwy 288 S Fwy SB approaching The S Sam Houston Tollway, stopped traffic back to Orem traffic',
  {'entities': [(26, 35, 'region'),
    (39, 93, 'relative location'),
    (111, 123, 'relative location'),
    (7, 22, 'delay reason')]}),
 ('Freeway closed due to a major accident. in Bellaire on 610 W Lp NB between Fournace and 69 The SW Fwy, stopped traffic back to Bellaire',
  {'entities': [(15, 38, 'delay reason'),
    (43, 51, 'region'),
    (52, 101, 'rela

In [39]:
import random

In [40]:
# train our spaCy model
# code adapted from creator of ner annotator
# https://medium.com/@manivannan_data/how-to-train-ner-with-custom-training-data-using-spacy-188e0e508c6

def train_spacy(data,iterations):
    training_data_ner = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in training_data_ner:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(training_data_ner)
            losses = {}
            for text, annotations in training_data_ner:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp






In [41]:
training_data_ner

[('Additional road closures in southeast Houston: 1) Airport between Telephone Road and Broadway 2) Mykawa between Dixie and Interstate Highway 610 3) Wayside between Dixie and Interstate Highway 610 4) Cullen at Almeda Genoa 5) Scott Street at Interstate Highway 610 houwx',
  {'entities': [(97, 144, 'relative location'),
    (50, 94, 'relative location'),
    (148, 196, 'relative location'),
    (226, 264, 'relative location'),
    (199, 222, 'relative location'),
    (28, 45, 'region')]}),
 ('Closed due to accident in SouthSide on Hwy 288 S Fwy SB approaching The S Sam Houston Tollway, stopped traffic back to Orem traffic',
  {'entities': [(26, 35, 'region'),
    (39, 93, 'relative location'),
    (111, 123, 'relative location'),
    (7, 22, 'delay reason')]}),
 ('Freeway closed due to a major accident. in Bellaire on 610 W Lp NB between Fournace and 69 The SW Fwy, stopped traffic back to Bellaire',
  {'entities': [(15, 38, 'delay reason'),
    (43, 51, 'region'),
    (52, 101, 'rela

In [65]:
model

'closures'

In [ ]:
## Extracting Locations from spaCy labels of tweets